<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
###################################

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeClassifier
from sklearn.preprocessing import StandardScaler

my_cols = ['ORGANIZATION_TYPE', 'CODE_GENDER', 'OCCUPATION_TYPE',
           'DAYS_BIRTH', 'AMT_INCOME_TOTAL', 'TARGET']
X = X_tr.iloc[:100][['CODE_GENDER', 'ORGANIZATION_TYPE', 'OCCUPATION_TYPE',
           'DAYS_BIRTH']]
y = X_tr.iloc[:100]['AMT_INCOME_TOTAL'] 
# X.shape, y.shape


pipe = Pipeline([
                 ('preproc', CustTransformer(thresh_card=8)),
	             ('reg', Ridge()) # ('clf', RidgeClassifier()) # classification
                ])


param_grid = {
              'preproc__strat_binary': ['ord'],
              'preproc__strat_low_card': ['ohe', 'ord'],
              'preproc__strat_high_card': ['ord', 'bin'],
              'preproc__strat_quant': ['stand', 'quant_norm'],
              'reg__alpha': np.logspace(-6,-9, 2)
                }

gs = GridSearchCV(estimator = pipe,
                  param_grid = param_grid,
                  scoring=['r2', 'neg_mean_absolute_error'], #['precision', 'recall', 'f1', 'roc_auc'],
                  refit='r2', #'roc_auc',
                  cv=4,
                  verbose=1,
                  return_train_score=True)

gs.fit(X, y)
# pipe.fit_predict(X)
# gs.best_params_

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_uniform('x', -5, 5)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100);

study.best_params, study.best_value, len(study.trials)#, study.trials, study.best_trial, 

study.optimize(objective, n_trials=100) # continue optimization
len(study.trials), study.best_params, study.best_value, # updated nb of trials

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.preprocessing import *

class CustTransformer(BaseEstimator) :

    def __init__(self, thresh_card=12,
                 strat_binary = 'ord', strat_low_card ='ohe',
                 strat_high_card ='bin', strat_quant = 'stand'):
        self.thresh_card = thresh_card
        self.strat_binary = strat_binary
        self.strat_low_card = strat_low_card
        self.strat_high_card = strat_high_card
        self.strat_quant = strat_quant

    def d_type_col(self, X):
        bin_cols = (X.nunique()[X.nunique()<=2].index)
        X_C_cols = X.select_dtypes(include=['object', 'category'])
        C_l_card_cols = X_C_cols.nunique()[X_C_cols.nunique().between(3, self.thresh_card)].index
        C_h_card_cols = X_C_cols.nunique()[X_C_cols.nunique()>self.thresh_card].index
        Q_cols = [c for c in X.select_dtypes(include=[np.number]).columns\
                                                        if c not in bin_cols]
        d_t = {'binary': bin_cols,
               'low_card': C_l_card_cols,
               'high_card': C_h_card_cols,
               'numeric': Q_cols}
        return d_t

    def get_feature_names(self):
        return self.ct_cat.get_feature_names()+self.num_cols

    def fit(self, X, y=None):
        # Dictionary to translate strategies
        d_enc = {'ohe': ce.OneHotEncoder(),
                 'hash': ce.HashingEncoder(),
                 'ord': ce.OrdinalEncoder(),
                 'loo': ce.LeaveOneOutEncoder(),
                 'bin': ce.BinaryEncoder(),
                 'stand': StandardScaler(),
                 'minmax': MinMaxScaler(),
                 'maxabs': MaxAbsScaler(),
                 'robust': RobustScaler(quantile_range=(25, 75)),
                 'norm': Normalizer(),
                 'quant_uni': QuantileTransformer(output_distribution='uniform'),
                 'quant_norm': QuantileTransformer(output_distribution='normal'),
                 'boxcox': PowerTransformer(method='boxcox'),
                 'yeo': PowerTransformer(method='yeo-johnson'),
                 'none': FunctionTransformer(func=lambda x:x,
                                             inverse_func=lambda x:x),
                 }

        self.ct_cat = ColumnTransformer([
                                        ('binary', d_enc[self.strat_binary], self.d_type_col(X)['binary']),
                                        ('low_card', d_enc[self.strat_low_card], self.d_type_col(X)['low_card']),
                                        ('high_card', d_enc[self.strat_high_card], self.d_type_col(X)['high_card']),
                                        ],
                                        #remainder='passthrough'
                                        )

        self.num_cols = self.d_type_col(X)['numeric']
        self.num_trans = Pipeline([("numeric", d_enc[self.strat_quant])])

        self.cat_cols = self.d_type_col(X)['binary'].union(self.d_type_col(X)['low_card']).union(self.d_type_col(X)['high_card'])
        self.cat_trans = Pipeline([("categ", self.ct_cat)])

        self.column_trans =  ColumnTransformer([
                                        ('cat', self.cat_trans, self.cat_cols),
                                        ('num', self.num_trans, self.num_cols),
                                        ],
                                        # remainder='passthrough'
                                        )
        self.ct_cat.fit(X, y)
        return self.column_trans.fit(X, y)
  
    def transform(self, X, y=None): # to use in a pipeline
        # return  self.column_trans.transform(X)
        return pd.DataFrame(self.column_trans.transform(X),
                            columns=self.get_feature_names())

    def fit_transform(self, X, y=None):
        # if y is None:
        #     self.fit(X)
        #     return self.column_trans.transform(X)
        # else:
        #     self.fit(X, y)
        #     return self.column_trans.transform(X)
        if y is None:  
            self.fit(X)
            return pd.DataFrame(self.column_trans.transform(X),
                            columns=self.get_feature_names())
        else: 
            self.fit(X, y)
            return pd.DataFrame(self.column_trans.transform(X),
                            columns=self.get_feature_names())
